In [1]:
import numpy as np
import torch

import matplotlib
import matplotlib.pyplot as plt
%matplotlib widget

import corner

import seaborn as sns
sns.set()

In [2]:
means = torch.Tensor([
    [0.0, 0.0],
    [2, 3],
    [2, -3]
])
covars = torch.Tensor([
    [
        [0.1, 0],
        [0, 1.5]
    ],
    [
        [1, 0],
        [0, 0.1]
    ],
    [
        [1, 0],
        [0, 0.1]
    ]
])

Z = torch.distributions.MultivariateNormal(loc=means, covariance_matrix=covars).sample((100000,)).reshape((-1, 2))
idx = torch.randperm(Z.shape[0])
Z = Z[idx]
Z_train = Z[:Z.shape[0] // 2]
Z_test = Z[Z.shape[0] // 2:]

x_lim = (-2, 6)
y_lim = (-6, 6)

fig, ax = plt.subplots()
corner.hist2d(Z_train[:, 0].numpy(), Z_train[:, 1].numpy(), ax=ax)
ax.set_xlim(x_lim)
ax.set_ylim(y_lim)
ax.set_title(r'$P(\mathbf{z})$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '$P(\\mathbf{z})$')

In [3]:
S = torch.Tensor([
    [0.1, 0],
    [0, 3]
])
X = Z + torch.distributions.MultivariateNormal(loc=torch.Tensor([0.0, 0.0]), covariance_matrix=S).sample((Z.shape[0],))
X_train = X[:X.shape[0] // 2]
X_test = X[X.shape[0] // 2:]

fig, ax = plt.subplots()
corner.hist2d(X_train[:, 0].numpy(), X_train[:, 1].numpy())
ax.set_xlim(x_lim)
ax.set_ylim(y_lim)
ax.set_title(r'$P(\mathbf{x})$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '$P(\\mathbf{x})$')

In [4]:
from deconv.gmm.data import DeconvDataset

train_data = DeconvDataset(X_train, torch.cholesky(S.repeat(X_train.shape[0], 1, 1)))
test_data = DeconvDataset(X_test, torch.cholesky(S.repeat(X_test.shape[0], 1, 1)))

In [5]:
from deconv.gmm.sgd_deconv_gmm import SGDDeconvGMM

gmm = SGDDeconvGMM(
    3,
    2,
    batch_size=200,
    lr=1e-1,
    epochs=100,
    device=torch.device('cuda')
)
gmm.fit(train_data, verbose=True)

Stating minibatch_k_means
Iter: 0
Iter: 1
Iter: 2


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/s0904254/miniconda3/envs/deconv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-c4aeef55f169>", line 11, in <module>
    gmm.fit(train_data, verbose=True)
  File "/mnt/cdtds_cluster_home/s0904254/xd/extreme-deconvolution/deconv/gmm/sgd_gmm.py", line 150, in fit
    self.init_params(loader)
  File "/mnt/cdtds_cluster_home/s0904254/xd/extreme-deconvolution/deconv/gmm/sgd_gmm.py", line 243, in init_params
    counts, centroids = minibatch_k_means(loader, self.k, max_iters=self.k_means_iters, device=self.device)
  File "/mnt/cdtds_cluster_home/s0904254/xd/extreme-deconvolution/deconv/gmm/util.py", line 50, in minibatch_k_means
    for d in loader:
  File "/home/s0904254/miniconda3/envs/deconv/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 345, in __next__
    data = self._next_data()
  File "/home/s0904254/m

KeyboardInterrupt: 

In [6]:
S.shape

torch.Size([2, 2])

In [12]:
from deconv.flow.svi import SVIFlow
svi = SVIFlow(
    2,
    5,
    device=torch.device('cuda'),
    batch_size=512,
    epochs=20,
    lr=1e-4,
    n_samples=50,
    use_iwae=False
)

In [20]:
svi.fit(train_data, val_data=None)

Epoch 0, Train Loss: -4.098158260091146
Epoch 1, Train Loss: -4.092337828776041
Epoch 2, Train Loss: -4.092128470052083
Epoch 3, Train Loss: -4.091051175944011
Epoch 4, Train Loss: -4.089471376953125
Epoch 5, Train Loss: -4.090663575032552
Epoch 6, Train Loss: -4.089579794108073
Epoch 7, Train Loss: -4.0890384049479165
Epoch 8, Train Loss: -4.089071686197917
Epoch 9, Train Loss: -4.088573410644531
Epoch 10, Train Loss: -4.08755037109375
Epoch 11, Train Loss: -4.087555048014323
Epoch 12, Train Loss: -4.0872225813802086
Epoch 13, Train Loss: -4.086628002929688
Epoch 14, Train Loss: -4.086408982747396
Epoch 15, Train Loss: -4.087736219889323
Epoch 16, Train Loss: -4.08664162109375
Epoch 17, Train Loss: -4.085366465657552
Epoch 18, Train Loss: -4.086735221354167
Epoch 19, Train Loss: -4.0859524690755205


In [18]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
prior_samples = svi.sample_prior(10000)

fig, ax = plt.subplots()
corner.hist2d(prior_samples[0, :, 0].numpy(), prior_samples[0, :, 1].numpy(), ax=ax)
ax.set_xlim(x_lim)
ax.set_ylim(y_lim)
ax.set_title('Prior fitted with ELBO')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Prior fitted with ELBO')

In [19]:
mean = np.array([[3.0, 0.0], [0.0, 0.0]])
cov = np.array([
    [
        [0.1, 0],
        [0, 3]
    ],
    [
        [0.1, 0],
        [0, 3]
    ]
])
test_point = [
    torch.Tensor(mean).to(svi.device),
    torch.cholesky(torch.Tensor(cov)).to(svi.device)
]
torch.set_default_tensor_type('torch.cuda.FloatTensor')
posterior_samples = svi.resample_posterior(test_point, 10000)

from deconv.gmm.plotting import plot_covariance
fig, ax = plt.subplots()
corner.hist2d(posterior_samples[0, :, 0].numpy(), posterior_samples[0, :, 1].numpy(), ax=ax)
ax.set_xlim(x_lim)
ax.set_ylim(y_lim)
plot_covariance(
    mean[0],
    cov[0],
    ax=ax,
    color='r'
)
ax.set_xlim(x_lim)
ax.set_ylim(y_lim)
ax.set_title('Recognition network samples for test point, fitted using ELBO.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Recognition network samples for test point, fitted using ELBO.')

In [ ]:
rsamples = svi.resample_posterior(test_point, 10000)

fig, ax = plt.subplots()
corner.hist2d(rsamples[0, :, 0].cpu().numpy(), rsamples[0, :, 1].cpu().numpy(), ax=ax)
ax.set_xlim(x_lim)
ax.set_ylim(y_lim)
plot_covariance(
    mean[0],
    cov[0],
    ax=ax,
    color='r'
)
ax.set_xlim(x_lim)
ax.set_ylim(y_lim)
ax.set_title('Resampled posterior for test point, fitted using ELBO.')